# Setup

In [72]:
!git clone https://github.com/H-TayyarMadabushi/SemEval_2022_Task2-idiomaticity.git
!git clone https://github.com/H-TayyarMadabushi/AStitchInLanguageModels.git
!git clone https://github.com/huggingface/transformers.git

fatal: destination path 'SemEval_2022_Task2-idiomaticity' already exists and is not an empty directory.
fatal: destination path 'AStitchInLanguageModels' already exists and is not an empty directory.
fatal: destination path 'transformers' already exists and is not an empty directory.


# Imports

In [2]:
import os
import csv
import sys
import random
import pickle
import logging
from pathlib import Path

import numpy as np
import pandas as pd
from sklearn.metrics import f1_score, accuracy_score

# Data Loading

In [3]:
directory   = 'SemEval_2022_Task2-idiomaticity/SubTaskA/Data/'
zero_shot_tr = os.path.join(directory, 'train_zero_shot.csv')
one_shot_tr = os.path.join(directory, 'train_one_shot.csv')
dev = os.path.join(directory, 'dev.csv')
dev_gold = os.path.join(directory, 'dev_gold.csv')

zero_shot_df = pd.read_csv(zero_shot_tr)
one_shot_df = pd.read_csv(one_shot_tr)
dev_df = pd.read_csv(dev)
dev_gold_df = pd.read_csv(dev_gold)
dev_df = dev_df.merge(dev_gold_df, validate='one_to_one')
dfs = {'Zero Shot Data': zero_shot_df, 'One Shot Data': one_shot_df, 'Dev Data': dev_df}

# EDA

In [4]:
display(zero_shot_df)
display(one_shot_df)
display(dev_df)

,DataID,Language,MWE,Setting,Previous,Target,Next,Label
0,train_zero_shot.EN.168.1,EN,double dutch,zero_shot,This inspired others to jump ropes as a leisur...,There are several theories behind the origin o...,The most popular theory states that “Double Du...,0
1,train_zero_shot.EN.168.2,EN,double dutch,zero_shot,In the age of chivalry a man paid for the woma...,"Double Dutch also derives from the same era, D...",There are many phrases that include the word: ...,0
2,train_zero_shot.EN.168.3,EN,double dutch,zero_shot,"To her eternal credit, she kept both India and...",Since 1977 we have had a plethora of Foreign M...,We need to exclude from that list the late Mr ...,0
3,train_zero_shot.EN.168.4,EN,double dutch,zero_shot,While pharmaceutical companies were researchin...,Turns out that these people were speaking doub...,So why aren’t Big Macs sold all over the world...,0
4,train_zero_shot.EN.168.5,EN,double dutch,zero_shot,Coronavirus in Europe * Brexit * Brussels ...,Is Flemish premier talking double Dutch?,Three months before the Belgians take over the...,0
...,...,...,...,...,...,...,...,...
4486,train_zero_shot.PT.351.8,PT,gato-pingado,zero_shot,Era o ano de 1968.,"A estação de passageiros tranquila, com um ou...","Com os primeiros raios solares, o potente DC 8...",0
4487,train_zero_shot.PT.351.9,PT,gato-pingado,zero_shot,De acordo com informações vazadas de dentro da...,"Segundo a fonte, o programa vai seguir um form...","Leonardo fica inchado, perde a voz e vício mor...",0
4488,train_zero_shot.PT.351.10,PT,gato-pingado,zero_shot,O radialista Roberto Toledo apresentou o show ...,"""Eu fiquei sentado no chão e realmente só tinh...",Entre os poucos espectadores estava o advogado...,0
4489,train_zero_shot.PT.351.11,PT,gato-pingado,zero_shot,O carnaval em Goiânia também tem espaço para o...,"É o caso do bloco “Gato Pingado”, que reuniu c...","Segundo os organizadores, quando criaram o blo...",1


,DataID,Language,MWE,Setting,Previous,Target,Next,Label
0,train_one_shot.EN.147.1,EN,high life,one_shot,PSG ace Marquinhos owns this £170k Ferrari - b...,Despite having the riches to afford the high l...,In an interview with the French magazine 'So F...,1
1,train_one_shot.EN.183.1,EN,birth rate,one_shot,Demography expert Piotr Szukalski told Dzienni...,Minister of Family and Social Policy Marlena M...,Commenting on data the state agency Statistics...,1
2,train_one_shot.EN.213.1,EN,home run,one_shot,Negro Leagues Baseball Museum President Bob Ke...,So Aaron faced the same brutal racism other Bl...,This Black man in the deep South was about to ...,0
3,train_one_shot.EN.26.1,EN,public service,one_shot,The Nakayama Public Service Scholarship is par...,Program leaders said the scholarship defines p...,I think everyone can agree we need to do more ...,0
4,train_one_shot.EN.26.2,EN,public service,one_shot,He was absolutely right.”,"In the ensuing years, Wennberg might not have ...",Wennberg said he knew when he ran for his sixt...,1
...,...,...,...,...,...,...,...,...
135,train_one_shot.PT.293.1,PT,amigo oculto,one_shot,"Mesmo com a pandemia, é possível manter a trad...",Chegou o fim do ano e com ele as festas de Nat...,"Não tem jeito, essa é uma tradição brasileira.",0
136,train_one_shot.PT.293.2,PT,amigo oculto,one_shot,Secret Santa Generator : Secret Santa é o nome...,Esta é a versão do app semelhante ao Amigo Ocu...,Eles são análogos e funcionam exatamente da me...,1
137,train_one_shot.PT.284.1,PT,agente secreto,one_shot,"Já à meia-noite, o Domingo Maior exibe 3 Dias ...","Ethan Renner (Kevin Costner), veterano agente ...","Seu último desejo é reatar com sua filha, com ...",1
138,train_one_shot.PT.435.1,PT,sangue frio,one_shot,"Com a pele em chamas, o companheiro de Khen (P...","Irritado com a mentira, o líder desejará matar...","Assim, ele decidirá se livrar do escolhido e l...",0


,ID,Language,MWE,Previous,Target,Next,DataID,Label
0,3652,EN,high life,"Does the plumbing predictably rebel, creating ...",Are these interruptions of the good life a nec...,"Not at all , says James von Klemperer, preside...",dev.EN.147.1,1
1,11103,EN,high life,Let’s be honest – we would be chuffed if our b...,But for Australian fashion designer Abby Kheir...,"Speaking to the Daily Mail , the owner of Abys...",dev.EN.147.2,1
2,84346,EN,high life,I already have the winning ticket.,"With that, I will be enjoying the pleasures of...","Charles Selle is a former News-Sun reporter, p...",dev.EN.147.3,1
3,56279,EN,high life,"There were signs everywhere of cosy, comfortab...","Fendi offered swaddling, belted coats resembli...",Shawl collars and enveloping dressing-gown sty...,dev.EN.147.4,1
4,17886,EN,high life,Yet one thing is clear.,"Rick Ross and Diddy exemplify the high life, t...",And who knows -- maybe we'll see Puff on his a...,dev.EN.147.5,1
...,...,...,...,...,...,...,...,...
734,95336,PT,papel higiênico,"O presidente da Suzano Papel e Celulose, Walte...",A Suzano Papel e Celulose é uma das 10 maiores...,Vale destacar que a Suzano já recebeu menos en...,dev.PT.394.4,1
735,20353,PT,papel higiênico,"Atualmente, poderiam surgir problemas com carg...","O presidente da Suzano Papel e Celulose, Walte...",A Suzano Papel e Celulose é uma das 10 maiores...,dev.PT.394.5,1
736,54479,PT,papel higiênico,Policiais penais do Complexo Penitenciário Dr ...,O material entorpecente estava escondido em ro...,O flagrante ocorreu na manhã desta sexta-feira...,dev.PT.394.6,1
737,85941,PT,papel higiênico,O acidente ocorreu depois de fortes rajadas de...,"No caso do papel higiênico, a escassez de cont...","Schalka, da Suzano, teme que os problemas logí...",dev.PT.394.7,1


In [66]:
for key, df in dfs.items():
    print("\n"+"#"*((40 - len(key))//2)+" {} ".format(key)+"#"*((40 - len(key))//2))
    print("\nTotal Samples: {}".format(len(df)))
    for col in ["Language", "Label", "MWE"]:
        value_counts = df[col].value_counts()
        unique_values = len(value_counts)
        if len(value_counts) > 5:
            print("\n{}".format(value_counts.iloc[:3].append(value_counts[-3:])))
        else:
            print("\n{}".format(value_counts))
        print("Number of {}s: {}".format(col, unique_values))


############# Zero Shot Data #############

Total Samples: 4491

EN    3327
PT    1164
Name: Language, dtype: int64
Number of Languages: 2

0    2535
1    1956
Name: Label, dtype: int64
Number of Labels: 2

information age     34
brass ring          32
peace talk          31
ônibus executivo     9
double dutch         9
algodão-doce         6
Name: MWE, dtype: int64
Number of MWEs: 236

############# One Shot Data #############

Total Samples: 140

EN    87
PT    53
Name: Language, dtype: int64
Number of Languages: 2

1    80
0    60
Name: Label, dtype: int64
Number of Labels: 2

sangue frio         2
dry land            2
closed book         2
goose egg           1
peace conference    1
entrance hall       1
Name: MWE, dtype: int64
Number of MWEs: 100

################ Dev Data ################

Total Samples: 739

EN    466
PT    273
Name: Language, dtype: int64
Number of Languages: 2

1    403
0    336
Name: Label, dtype: int64
Number of Labels: 2

dust storm         25
sangue quen

In [73]:
zero_shot_unique_eng_mwes = len(set(zero_shot_df[zero_shot_df["Language"] == 'EN']["MWE"]))
zero_shot_unique_pt_mwes = len(set(zero_shot_df[zero_shot_df["Language"] == 'PT']["MWE"]))
one_shot_unique_eng_mwes = len(set(one_shot_df[one_shot_df["Language"] == 'EN']["MWE"]))
one_shot_unique_pt_mwes = len(set(one_shot_df[one_shot_df["Language"] == 'PT']["MWE"]))
print("Number of English MWEs in zero shot data: {}".format(zero_shot_unique_eng_mwes))
print("Number of Portuguese MWEs in zero shot data: {}".format(zero_shot_unique_pt_mwes))
print("Number of English MWEs in one shot data: {}".format(one_shot_unique_eng_mwes))
print("Number of Portuguese MWEs in one shot data: {}".format(one_shot_unique_pt_mwes))

Number of English MWEs in zero shot data: 163
Number of Portuguese MWEs in zero shot data: 73
Number of English MWEs in one shot data: 60
Number of Portuguese MWEs in one shot data: 40


In [9]:
common_zero_one = set(zero_shot_df['MWE']) & set(one_shot_df['MWE'])
common_zero_dev = set(zero_shot_df['MWE']) & set(dev_df['MWE'])
common_one_dev = set(one_shot_df['MWE']) & set(dev_df['MWE'])
print("Number of common MWEs in zero and one shot data: {}".format(len(common_zero_one)))
print("Number of common MWEs in zero shot and dev data: {}".format(len(common_zero_dev)))
print("Number of common MWEs in one shot and dev data: {}".format(len(common_one_dev)))

Number of common MWEs in zero and one shot data: 0
Number of common MWEs in zero shot and dev data: 0
Number of common MWEs in one shot and dev data: 50


All dev MWEs are available in one shot data. There is no overlap of zero shot MWEs with one shot MWEs (and hence, no overlap with dev MWEs)

In [71]:
dataid_grouped_by_mwe_label = one_shot_df.groupby(["MWE", "Label"])["DataID"]
num_repeated_mwe_label_samples = len(dataid_grouped_by_mwe_label.filter(lambda x: x.count() != 1))
print("Cases with more than one sample with same MWE and Label in one shot data: {}".format(num_repeated_mwe_label_samples))

Cases with more than one sample with same MWE and Label in one shot data: 0


There is exactly one sample of each MWE-Label pair in one shot data

In [59]:
grouped_label = one_shot_df.groupby("MWE")["Label"]
two_unique_label_mwes = grouped_label.filter(lambda x: len(x.value_counts())==2)
print("Number of one shot MWEs with both labels: {}".format(len(two_unique_label_mwes) // 2))
one_unique_label_mwes = grouped_label.filter(lambda x: len(x.value_counts())==1)
zero_as_label_mwes = one_unique_label_mwes[one_unique_label_mwes==0]
one_as_label_mwes = one_unique_label_mwes[one_unique_label_mwes==1]
print("Number of one shot MWEs with only '1' as the label (only non idiomatic): {}".format(len(one_as_label_mwes)))
print("Number of one shot MWEs with only '0' as the label (only idiomatic): {}".format(len(zero_as_label_mwes)))

Number of one shot MWEs with both labels: 40
Number of one shot MWEs with only '1' as the label (only non idiomatic): 40
Number of one shot MWEs with only '0' as the label (only idiomatic): 20


Not all one shot MWEs have both samples. Some have only non idiomatic (40%) and some have only idiomatic (20%)

In [58]:
one_shot_in_dev_df = one_shot_df[one_shot_df["MWE"].isin(set(dev_df["MWE"]))]
grouped_label = one_shot_in_dev_df.groupby("MWE")["Label"]
two_unique_label_mwes = grouped_label.filter(lambda x: len(x.value_counts())==2)
print("Number of dev MWEs with both labels in one shot data: {}".format(len(two_unique_label_mwes) // 2))
one_unique_label_mwes = grouped_label.filter(lambda x: len(x.value_counts())==1)
zero_as_label_mwes = one_unique_label_mwes[one_unique_label_mwes==0]
one_as_label_mwes = one_unique_label_mwes[one_unique_label_mwes==1]
print("Number of dev MWEs with only '1' as the label in one shot data (only non idiomatic): {}".format(len(one_as_label_mwes)))
print("Number of dev MWEs with only '0' as the label in one shot data (only idiomatic): {}".format(len(zero_as_label_mwes)))

Number of dev MWEs with both labels in one shot data: 21
Number of dev MWEs with only '1' as the label in one shot data (only non idiomatic): 17
Number of dev MWEs with only '0' as the label in one shot data (only idiomatic): 12


Authors claim in the paper that all dev MWEs have atleast a non idiomatic samples in one shot data. That's probably not true. All dev MWEs have atleast one sample in one shot data, but that's not necessarily non idiomatic. 12/50 unique MWEs have only idiomatic one shot sample.